In [ ]:
from ultralytics import YOLO
import numpy as np
import cv2
import matplotlib.pyplot as plt
from src.mapping_utils.keypoint_coordinates import *
import os

In [ ]:
image_folder = 'data/1_train-val_1min_aalesund_from_start/img1'

keypoint_model = YOLO('yolo_weights/keypoint_weights.pt')

image_folder = sorted([os.path.join(image_folder, f) for f in os.listdir(image_folder) if f.endswith('.jpg')])

#### Keypoint model

In [ ]:
for image_path in image_folder:
    print(image_path)
    for result in keypoint_model.track(image_path, persist=True, stream=True, line_width=1, tracker='botsort.yaml'):
        annotated_frame = result.plot(font_size=1, line_width=1)
        cv2.imshow('frame', annotated_frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27:
            break
    if key == 27:
        break
cv2.destroyAllWindows()    

In [ ]:
#TODO add a function that chooses the 4 keypoints with the best accuracy

### Initiating football field coordinates

In [ ]:
football_field_coordinates = {
    1: [ 45. , 565. ], # 1
    2: [ 457.5, 565.], # 2
    3: [ 45. ,  30. ], # 3
    4: [ 457.5, 30. ], # 4
    5: [ 45. , 455. ], # 5
    6: [175. , 455. ], # 6
    7: [175. , 350. ], # 7
    8: [175. , 250. ], # 8
    9: [175. , 140. ], # 9
    10: [ 45. , 140. ], # 10
    11: [ 45. , 370. ], # 11
    12: [ 85. , 370. ], # 12 
    13: [ 85. , 230. ], # 13
    14: [ 45  , 230. ], # 14
    15: [130. , 300. ], # 15
    16: [457.5, 370. ], # 16 
    17: [457.5, 300. ], # 17
    18: [457.5, 225. ], # 18
    19: [200. , 300. ], # 19
    20: [385. , 300. ], # 20 
    21: [530. , 300. ], # 21 
    22: [870. , 565. ], # 22
    23: [870. ,  30. ], # 23
    25: [870. , 455. ], # 25 
    26: [740. , 455. ], # 26
    27: [740. , 350. ], # 27
    28: [740. , 250. ], # 28
    29: [740. , 140. ], # 29
    30: [870. , 140. ], # 30
    31: [870. , 370. ], # 31
    32: [830. , 370. ], # 32
    33: [830. , 230. ], # 33
    34: [870  , 230. ], # 34
    35: [785. , 300. ], # 35
    39: [715. , 300. ]  # 39
}
# Convert football_field_coordinates to a dictionary
football_field_coordinates_dict = {i+1: coord for i, coord in enumerate(football_field_coordinates)}

keypoint_mirror_mapping = {
    0: 18, 2: 19, 3: 20, 4: 21, 5: 22, 6: 23, 7: 24,
    8: 25, 9: 26, 15: 27, 16: 17
}

In [ ]:
# Save the dictionary to a text file
with open('football_field_coordinates.txt', 'w') as file:
    for key, value in football_field_coordinates.items():
        file.write(f'{key}: {value}\n')

### Initiating model

In [ ]:
image_path = 'data/1_train-val_1min_aalesund_from_start/img1/000041.jpg'

In [ ]:
keypoint_results = keypoint_model(image_path)

keypoint_confidence = keypoint_results[0].boxes.conf.cpu().numpy()
class_id = keypoint_results[0].boxes.cls.cpu().numpy() + 1
bounding_boxes = keypoint_results[0].boxes.xyxy.cpu().numpy()

top_four_indexes = np.argsort(-keypoint_confidence)[:4]

top_class_id = class_id[top_four_indexes]
top_bounding_boxes = bounding_boxes[top_four_indexes]

top_centers = np.array([[0.5 * (box[0] + box[2]), box[1] ] for box in top_bounding_boxes])

real_centers = []

for i, center in enumerate(top_centers):
    current_class_id = int(top_class_id[i])
    if current_class_id in football_field_coordinates:
        real_centers.append(football_field_coordinates[current_class_id])
    else:
        real_centers.append(center.tolist())

In [ ]:
H, status = cv2.findHomography(top_centers, np.array(real_centers))

In [ ]:
combined_model = YOLO('yolo_weights/combined_weights.pt')
combined_results = combined_model(image_path)

bounding_boxes = combined_results[0].boxes.xyxy.cpu().numpy()
centers = np.array([[0.5 * (box[0] + box[2]), 0.5 * (box[1] + box[3])] for box in bounding_boxes])

In [ ]:
field_coordinates = cv2.perspectiveTransform(centers.reshape(-1, 1, 2), H).reshape(-1, 2)
draw_football_field(field_coordinates)